#### Setup the notebook

In [3]:
#************************************************************************************************
# Imports
#************************************************************************************************
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

#************************************************************************************************
# Load environment variables from a .env file
#************************************************************************************************
load_dotenv()
#************************************************************************************************
# Create client instance of the AzureOpenAI class
#************************************************************************************************
AOAI = AzureOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        )


#### Create the assistant

In [4]:
#************************************************************************************************
# Create an assistant
#************************************************************************************************
assistant = AOAI.beta.assistants.create(
    name="Rulemaking Assistant",
    instructions="""**Persona**
You are an expert rulemaking assistant skilled at responding to questions about government regulations and using the regulations.gov API. You write concisely and clearly, in plain language, responding directly to users.

**Situation**
Users will ask you questions about proposed government rules. You will use your tools and knowledge to provide thorough and well-supported answers.

**Tools Available**
### get_list_of_dockets - Use this tool to get a list of dockets from regulations.gov.
- Parameters: 
    - agencyId: Filters results for the agency acronym specified in the value. Example: 'EPA'
    - searchTerm: Filters results on the given term.
    - postedDate: Filters results for dockets posted on the date. Format of date must be yyyy-mm-dd. 
    - beforePostedDate: Filters results for dockets posted on or before beforePostedDate. Format of the date MUST be yyyy-mm-dd.
    - afterPostedDate: Filters results for dockets posted on or after afterPostedDate. Format of the date MUST be yyyy-mm-dd.
- Returns: A list of dockets with the following fields:
    - id: The docket ID.
    - attributes: The attributes of the docket.
- Notes:
    - At least one of the parameters must be provided.
    - If postedDate is provided, beforePostedDate and afterPostedDate should not be provided.
    - For a date range, use beforePostedDate and afterPostedDate.

### get_docket_details - Use this tool to get the detailed information of a specific docketId from regulations.gov.
- Parameters:
    - docketId: The ID of the docket.
- Returns: The detailed information about the docket including the content of the prposed rule.
""",
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
)

# Save the assistant id
assistant_id = assistant.id


#### Give the assistant tools

In [6]:
# Define the JSON for the function

function_json_1 = {
    "name": "get_list_of_dockets",
    "description": "Retrieves a list of dockets.",
    "parameters": {
        "type": "object",
        "properties": {
            "agencyId": {
                "type": "string",
                "description": "The agency acronym to filter results. Example: 'EPA'"
            },
            "searchTerm": {
                "type": "string",
                "description": "The term to filter results for."
            },
            "postedDate": {
                "type": "string",
                "description": "Filters results for dockets posted on the date. Format of date must be yyyy-mm-dd."
            },
            "beforePostedDate": {
                "type": "string",
                "description": "Filters results for dockets posted on or before beforePostedDate. Format of the date MUST be yyyy-mm-dd."
            },
            "afterPostedDate": {
                "type": "string",
                "description": "Filters results for dockets posted on or after afterPostedDate. Format of the date MUST be yyyy-mm"
            }
        },
        "required": [""]
    }
}

function_json_2 = {
    "name": "get_docket_details",
    "description": "Given a specific docucmentId (docketId) this retrieves the docket content.",
    "parameters": {
        "type": "object",
        "properties": {
            "docketId": {
                "type": "string",
                "description": "The ID of the docket."
            }
        },
        "required": ["docketId"]
    }
}

function_json_3 = {
    "name": "stakeholder_analysis",
    "description": "Given a speccifc Document Id (docketId) this returns analsis of the stakeholders.",
    "parameters": {
        "type": "object",
        "properties": {
            "docketId": {
                "type": "string",
                "description": "The ID of the docket."
            }
        },
        "required": ["docketId"]
    }
}



# Update the assistant with the function 
assistant = AOAI.beta.assistants.update(
    assistant_id,
    tools=[
        {"type": "code_interpreter"},
        {"type": "function", "function": function_json_1},
        {"type": "function", "function": function_json_2},
    ]
)

# Show the assistant object
display(json.loads(assistant.model_dump_json()))

{'id': 'asst_JLRYLt14l5wWRbpC6Y1VXwbc',
 'created_at': 1714703098,
 'description': None,
 'file_ids': [],
 'instructions': "**Persona**\nYou are an expert rulemaking assistant skilled at responding to questions about government regulations and using the regulations.gov API. You write concisely and clearly, in plain language, responding directly to users.\n\n**Situation**\nUsers will ask you questions about proposed government rules. You will use your tools and knowledge to provide thorough and well-supported answers.\n\n**Tools Available**\n### get_list_of_dockets - Use this tool to get a list of dockets from regulations.gov.\n- Parameters: \n    - agencyId: Filters results for the agency acronym specified in the value. Example: 'EPA'\n    - searchTerm: Filters results on the given term.\n    - postedDate: Filters results for dockets posted on the date. Format of date must be yyyy-mm-dd. \n    - beforePostedDate: Filters results for dockets posted on or before beforePostedDate. Format 